## Парсинг URL ("Риа Новости")

In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [2]:
import tqdm
import json
import re
import math

In [3]:
chrome_driver_path = "/home/arklykov/chromedriver.exe"

In [15]:
prefs = {"profile.managed_default_content_settings.images": 2,
             "profile.default_content_settings.cookies": 2}
chrome_options = Options()
chrome_options.add_extension('education/adblock.crx')
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_argument('--disable-application-cache')
driver = Chrome(executable_path=chrome_driver_path, chrome_options=chrome_options)
driver.implicitly_wait(10)
print("Current session is {}".format(driver.session_id))
driver.switch_to.window(driver.window_handles[0])
driver.get('chrome-extension://gighmmpiobklfepjocnamgkkbiglidom/options.html')
driver.find_element(By.ID, 'acceptable_ads').click()

/tmp/ipykernel_47173/2512148171.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome(executable_path=chrome_driver_path, chrome_options=chrome_options)
/tmp/ipykernel_47173/2512148171.py:7: DeprecationWarning: use options instead of chrome_options
  driver = Chrome(executable_path=chrome_driver_path, chrome_options=chrome_options)


Current session is 06f026ae8ebd9c2960721af3f4e53b63


In [4]:
categories = ['economy', 'science', 'culture']
sport_categories = ['football', 'hockey', 'tennis', 'figure_skating', 'fights', 'category_formula_1', 'biathlon', 'zozh']

In [5]:
links_filename = "files/links.txt"

In [6]:
def parce_links_categories(category, output):
    driver.get(f"https://ria.ru/{category}/")
    load_elements = re.findall(r'\d+', driver.find_element(By.CLASS_NAME, 'list-more').text)
    load_elements = int(load_elements[0])
    for element in driver.find_elements(By.CLASS_NAME, 'list-item'):
        with open(output,'a') as file:
            file.write(json.dumps({
                'link' : element.find_element(By.CLASS_NAME, 'list-item__image').get_attribute('href'),
                'category': category
            },ensure_ascii=False) + '\n')
    new_url = driver.find_element(By.CLASS_NAME, 'list-more').get_attribute('data-url')

    steps = math.ceil(1000 / load_elements)
    for step in range(steps):
        driver.get(f"https://ria.ru{new_url}")

        for element in driver.find_elements(By.CLASS_NAME, 'list-item'):
            with open(output,'a') as file:
                file.write(json.dumps({
                    'link' : element.find_element(By.CLASS_NAME, 'list-item__image').get_attribute('href'),
                    'category': category
                },ensure_ascii=False) + '\n')
        new_url = driver.find_element(By.CLASS_NAME, 'list-items-loaded').get_attribute('data-next-url')    

In [7]:
def parce_links_sport_categories(category, output):
    driver.get(f"https://rsport.ria.ru/{category}")
    load_elements = re.findall(r'\d+', driver.find_element(By.CLASS_NAME, 'list-more').text)
    load_elements = int(load_elements[0])
    for element in driver.find_elements(By.CLASS_NAME, 'list-item'):
        with open(output,'a') as file:
            file.write(json.dumps({
                'link' : element.find_element(By.CLASS_NAME, 'list-item__image').get_attribute('href'),
                'category': 'sport'
            },ensure_ascii=False) + '\n')
    new_url = driver.find_element(By.CLASS_NAME, 'list-more').get_attribute('data-url')

    steps = math.ceil(125 / load_elements)
    for step in range(steps):
        driver.get(f"https://rsport.ria.ru{new_url}")

        for element in driver.find_elements(By.CLASS_NAME, 'list-item'):
            with open(output,'a') as file:
                file.write(json.dumps({
                    'link' : element.find_element(By.CLASS_NAME, 'list-item__image').get_attribute('href'),
                    'category': 'sport'
                },ensure_ascii=False) + '\n')
        new_url = driver.find_element(By.CLASS_NAME, 'list-items-loaded').get_attribute('data-next-url')

In [159]:
for category in categories:
    parce_links_categories(category, links_filename)

In [160]:
for sport_caterory in sport_categories:
    parce_links_sport_categories(sport_caterory, links_filename)

## Парсинг новостей с "Риа Новости"

In [77]:
import requests
from bs4 import BeautifulSoup
from html2text import html2text

In [78]:
catalog_filename = "files/catalog.txt"

In [79]:
posts = []
with open(links_filename, 'r') as file:
    for line in file:
        posts.append(json.loads(line))

In [82]:
def create_document_bs(post, output):
    article_id = post['link']
    category = post['category']

    bsoup = BeautifulSoup(requests.get(article_id).text, 'lxml')

    title = bsoup.find('meta', attrs={'name': 'analytics:title'}).get(key='content')

    tags = bsoup.find('meta', attrs={'name': 'analytics:tags'}).get(key='content')

    text = bsoup.find('div', class_='article__body').find_all('div', attrs={'data-type':'text'})
    text = [t.text for t in text]
    text = "".join(text)
    
    with open(output,'a') as file:
        file.write(json.dumps({
            "article_id": article_id,
            "title": title,
            "category": category,
            "tags": tags,
            "text": text
        },ensure_ascii=False) + '\n')

In [83]:
for post in tqdm.tqdm(posts):
    create_document_bs(post, catalog_filename)

100%|██████████| 4244/4244 [1:06:40<00:00,  1.06it/s]


In [84]:
catalog = []
with open(catalog_filename, 'r') as file:
    for line in file:
        catalog.append(json.loads(line))

In [90]:
corpus = {
    "catalog": catalog
}

In [93]:
corpus_filename = 'files/corpus.json'

In [92]:
with open(corpus_filename, 'w') as file:
    file.write(json.dumps(corpus, ensure_ascii=False) + '\n')

## Обзор данных

In [98]:
import pandas as pd

In [95]:
with open(corpus_filename, 'r') as file:
    corpus_ = json.load(file)

In [99]:
frame_corpus_ = pd.DataFrame(corpus_['catalog'])

In [102]:
frame_corpus_.head()

,article_id,title,category,tags,text
0,https://ria.ru/20230115/neft-1845019942.html,"Иран ограничит поставки дешевой нефти в Сирию,...",economy,"Экономика, Цены на нефть, Сирия, Иран, Дамаск ...","ВАШИНГТОН, 15 янв - РИА Новости. Иран ограничи..."
1,https://ria.ru/20230115/mestorozhdenie-1845012...,Итальянская нефтегазовая компания обнаружила н...,economy,"Экономика, Цены на нефть, Египет, Средиземное ...","РИМ, 15 янв - РИА Новости. Итальянская нефтега..."
2,https://ria.ru/20230115/inflyatsiya-1844997935...,Инфляция в России может приблизиться к пяти пр...,economy,"Россия, Экономика, Владимир Путин, Павел Зарубин","ПЕТРОПАВЛОВСК-КАМЧАТСКИЙ, 15 янв - РИА Новости..."
3,https://ria.ru/20230115/ekonomika-1844996743.html,Путин назвал ситуацию в российской экономике с...,economy,"Экономика, Цены на нефть, Владимир Путин, Паве...","ПЕТРОПАВЛОВСК-КАМЧАТСКИЙ, 15 янв — РИА Новости..."
4,https://ria.ru/20230115/neft-1844714708.html,Редкие сорта: азиаты скупают российскую арктич...,economy,"Экономика, Китай, Россия, Индия, Международное...","МОСКВА, 15 янв — РИА Новости, Наталья Дембинск..."
